In [1]:
import IPython.nbformat.current as nbf

C:\Users\Shyam\Anaconda3\lib\site-packages\IPython\nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
C:\Users\Shyam\Anaconda3\lib\site-packages\nbformat\current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [2]:
# %load AirlineANN.py
"""
Created on Sat Oct 21 16:29:15 2017

@author: Shyam
"""


import sys
import random
import numpy as np

from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
import sklearn.feature_selection
from sklearn.cross_validation import train_test_split
import sklearn.feature_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,confusion_matrix,classification_report

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


C:\Users\Shyam\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
mdata = pd.read_pickle('rawmodeldata.pkl')

In [4]:
data =mdata[['DepDelay', 'Year', 'Month', 'DayofMonth', 'DayOfWeek', 'hour', 'Distance', 'UniqueCarrier', 'Dest', 'd2h', 'isdelay', 'Origin']]


In [5]:
data = data.sample(frac = 0.5)

In [6]:
cols = ['Month', 'DayofMonth', 'DayOfWeek', 'hour', 'Distance', 'd2h','UniqueCarrier', 'Dest']
colsy= ['isdelay']

In [7]:
X = data[cols]

In [8]:
X.columns

Index(['Month', 'DayofMonth', 'DayOfWeek', 'hour', 'Distance', 'd2h',
       'UniqueCarrier', 'Dest'],
      dtype='object')

In [9]:
y=data[colsy]

In [10]:
weather = pd.read_csv('data/weather_2008.csv')

In [11]:
fdata = data.merge(weather, left_on=['Year','Month', 'DayofMonth', 'Origin'], right_on=['year','month','day','origin'])

In [12]:
fcols = ['Month', 'DayofMonth', 'DayOfWeek', 'hour','Distance', 'UniqueCarrier', 'Dest', 'd2h', 'isdelay',  'maxt', 'mint']
d_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'hour','Dest']

In [13]:
XX = fdata[fcols]

In [14]:
X=XX.drop('isdelay',axis=1)

In [15]:
y = XX[colsy]

In [16]:
import tensorflow as tf

In [17]:
Month = tf.feature_column.categorical_column_with_hash_bucket("Month",hash_bucket_size=12,dtype= tf.int64)
DayofMonth = tf.feature_column.categorical_column_with_hash_bucket("DayofMonth",hash_bucket_size=31,dtype= tf.int64)
DayOfWeek = tf.feature_column.categorical_column_with_hash_bucket("DayOfWeek",hash_bucket_size=10,dtype= tf.int64)
UniqueCarrier = tf.feature_column.categorical_column_with_hash_bucket("UniqueCarrier",hash_bucket_size=1000)
hour = tf.feature_column.categorical_column_with_hash_bucket("hour",hash_bucket_size=50,dtype= tf.int64)
Dest = tf.feature_column.categorical_column_with_hash_bucket("Dest",hash_bucket_size=1000)



In [18]:
Distance = tf.feature_column.numeric_column("Distance")
d2h = tf.feature_column.numeric_column("d2h")
maxt = tf.feature_column.numeric_column("maxt")
mint = tf.feature_column.numeric_column("mint")

In [19]:
feat_cols = [Month, DayofMonth, DayOfWeek, hour,Distance, UniqueCarrier, Dest, d2h, maxt, mint]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [21]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=1000,num_epochs=None,shuffle=True)

In [22]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Shyam\\AppData\\Local\\Temp\\tmpfui0o3yj', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [23]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Shyam\AppData\Local\Temp\tmpfui0o3yj\model.ckpt.
INFO:tensorflow:loss = 693.146, step = 1
INFO:tensorflow:global_step/sec: 68.8157
INFO:tensorflow:loss = 4536.43, step = 101 (1.453 sec)
INFO:tensorflow:global_step/sec: 74.4188
INFO:tensorflow:loss = 4122.95, step = 201 (1.344 sec)
INFO:tensorflow:global_step/sec: 69.5643
INFO:tensorflow:loss = 1308.6, step = 301 (1.438 sec)
INFO:tensorflow:global_step/sec: 57.1422
INFO:tensorflow:loss = 1611.41, step = 401 (1.750 sec)
INFO:tensorflow:global_step/sec: 70.3288
INFO:tensorflow:loss = 491.467, step = 501 (1.422 sec)
INFO:tensorflow:global_step/sec: 73.5617
INFO:tensorflow:loss = 1907.57, step = 601 (1.359 sec)
INFO:tensorflow:global_step/sec: 74.4185
INFO:tensorflow:loss = 1593.92, step = 701 (1.344 sec)
INFO:tensorflow:global_step/sec: 74.4177
INFO:tensorflow:loss = 502.004, step = 801 (1.344 sec)
INFO:tensorflow:global_step/sec: 75.2933
INF

INFO:tensorflow:global_step/sec: 53.2246
INFO:tensorflow:loss = 520.246, step = 8401 (1.816 sec)
INFO:tensorflow:global_step/sec: 67.3675
INFO:tensorflow:loss = 467.287, step = 8501 (1.484 sec)
INFO:tensorflow:global_step/sec: 78.0468
INFO:tensorflow:loss = 578.698, step = 8601 (1.281 sec)
INFO:tensorflow:global_step/sec: 78.0479
INFO:tensorflow:loss = 503.662, step = 8701 (1.281 sec)
INFO:tensorflow:global_step/sec: 78.049
INFO:tensorflow:loss = 457.707, step = 8801 (1.281 sec)
INFO:tensorflow:global_step/sec: 78.0468
INFO:tensorflow:loss = 479.679, step = 8901 (1.281 sec)
INFO:tensorflow:global_step/sec: 78.049
INFO:tensorflow:loss = 486.367, step = 9001 (1.297 sec)
INFO:tensorflow:global_step/sec: 78.0467
INFO:tensorflow:loss = 489.103, step = 9101 (1.266 sec)
INFO:tensorflow:global_step/sec: 78.049
INFO:tensorflow:loss = 686.205, step = 9201 (1.281 sec)
INFO:tensorflow:global_step/sec: 79.0114
INFO:tensorflow:loss = 507.194, step = 9301 (1.281 sec)
INFO:tensorflow:global_step/sec: 

In [24]:
print(model.params)

{}


In [25]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [26]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\Shyam\AppData\Local\Temp\tmpfui0o3yj\model.ckpt-10000


In [27]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [28]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.83      0.92      0.87    233056
          1       0.42      0.23      0.30     56833

avg / total       0.75      0.79      0.76    289889



In [29]:
print(roc_auc_score(y_test,final_preds))

0.576819326665


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [31]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [32]:
Month1 = tf.feature_column.categorical_column_with_hash_bucket("Month",hash_bucket_size=12,dtype= tf.int64)
DayofMonth1 = tf.feature_column.categorical_column_with_hash_bucket("DayofMonth",hash_bucket_size=31,dtype= tf.int64)
DayOfWeek1 = tf.feature_column.categorical_column_with_hash_bucket("DayOfWeek",hash_bucket_size=10,dtype= tf.int64)
UniqueCarrier1 = tf.feature_column.categorical_column_with_hash_bucket("UniqueCarrier",hash_bucket_size=1000)
hour1 = tf.feature_column.categorical_column_with_hash_bucket("hour",hash_bucket_size=50,dtype= tf.int64)
Dest1 = tf.feature_column.categorical_column_with_hash_bucket("Dest",hash_bucket_size=1000)


In [33]:
Month11 = tf.feature_column.embedding_column(Month1, dimension=12)
DayofMonth11 = tf.feature_column.embedding_column(DayofMonth1, dimension=31)
DayOfWeek11 = tf.feature_column.embedding_column(DayOfWeek1, dimension=10)
UniqueCarrier11 = tf.feature_column.embedding_column(UniqueCarrier1, dimension=1000)
hour11 = tf.feature_column.embedding_column(hour1, dimension=50)
Dest11 = tf.feature_column.embedding_column(Dest1, dimension=1000)

In [34]:
feat_cols1 = [Month11, DayofMonth11, DayOfWeek11, hour11,Distance, UniqueCarrier11, Dest11, d2h, maxt, mint]

In [35]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=200,num_epochs=200000,shuffle=True)

In [36]:
import warnings
warnings.filterwarnings('ignore')
import warnings; warnings.simplefilter('ignore')

In [41]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[50,30,10], dropout=0.5, feature_columns=feat_cols1,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Shyam\\AppData\\Local\\Temp\\tmpv76myz2z', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [42]:
dnn_model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Shyam\AppData\Local\Temp\tmpv76myz2z\model.ckpt.
INFO:tensorflow:loss = 3192.4, step = 1
INFO:tensorflow:global_step/sec: 52.4585
INFO:tensorflow:loss = 102.274, step = 101 (1.922 sec)
INFO:tensorflow:global_step/sec: 53.3327
INFO:tensorflow:loss = 99.658, step = 201 (1.859 sec)
INFO:tensorflow:global_step/sec: 50.7931
INFO:tensorflow:loss = 90.7261, step = 301 (1.984 sec)
INFO:tensorflow:global_step/sec: 49.2302
INFO:tensorflow:loss = 93.96, step = 401 (2.016 sec)
INFO:tensorflow:global_step/sec: 53.3327
INFO:tensorflow:loss = 101.759, step = 501 (1.891 sec)
INFO:tensorflow:global_step/sec: 54.7002
INFO:tensorflow:loss = 97.0043, step = 601 (1.813 sec)
INFO:tensorflow:global_step/sec: 54.2367
INFO:tensorflow:loss = 100.4, step = 701 (1.844 sec)
INFO:tensorflow:global_step/sec: 54.7002
INFO:tensorflow:loss = 94.8306, step = 801 (1.828 sec)
INFO:tensorflow:global_step/sec: 55.1717
INFO:ten

INFO:tensorflow:global_step/sec: 54.7002
INFO:tensorflow:loss = 114.193, step = 8401 (1.828 sec)
INFO:tensorflow:global_step/sec: 55.4038
INFO:tensorflow:loss = 114.202, step = 8501 (1.805 sec)
INFO:tensorflow:global_step/sec: 52.0319
INFO:tensorflow:loss = 97.2477, step = 8601 (1.922 sec)
INFO:tensorflow:global_step/sec: 50.3931
INFO:tensorflow:loss = 91.5835, step = 8701 (1.984 sec)
INFO:tensorflow:global_step/sec: 49.2302
INFO:tensorflow:loss = 112.763, step = 8801 (2.031 sec)
INFO:tensorflow:global_step/sec: 49.6118
INFO:tensorflow:loss = 105.717, step = 8901 (2.016 sec)
INFO:tensorflow:global_step/sec: 54.7002
INFO:tensorflow:loss = 90.2756, step = 9001 (1.828 sec)
INFO:tensorflow:global_step/sec: 55.6516
INFO:tensorflow:loss = 76.2399, step = 9101 (1.813 sec)
INFO:tensorflow:global_step/sec: 54.7002
INFO:tensorflow:loss = 97.25, step = 9201 (1.813 sec)
INFO:tensorflow:global_step/sec: 54.7002
INFO:tensorflow:loss = 95.8623, step = 9301 (1.844 sec)
INFO:tensorflow:global_step/sec:

In [43]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=200,
      num_epochs=1,
      shuffle=False)

In [44]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2017-10-23-08:06:44
INFO:tensorflow:Restoring parameters from C:\Users\Shyam\AppData\Local\Temp\tmpv76myz2z\model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2017-10-23-08:07:07
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.803949, accuracy_baseline = 0.803949, auc = 0.5, auc_precision_recall = 0.598025, average_loss = 0.494889, global_step = 10000, label/mean = 0.196051, loss = 98.9399, prediction/mean = 0.197869


{'accuracy': 0.80394912,
 'accuracy_baseline': 0.80394912,
 'auc': 0.5,
 'auc_precision_recall': 0.59802544,
 'average_loss': 0.49488884,
 'global_step': 10000,
 'label/mean': 0.1960509,
 'loss': 98.93988,
 'prediction/mean': 0.19786875}